In [1]:
import cnvpytor
import subprocess

print(cnvpytor.__version__)

1.3.1


In [2]:
filename = "test_with_del_400_2_formated.fa"
command = f"cat {filename} | wc -l"
result = subprocess.check_output(command, shell=True)
line_count = int(result.strip())
line_count

1063477

In [3]:
read_count = line_count * 50 * 30 / 202
read_count = round(read_count)
read_count

7897106

In [4]:
command = f"wgsim -d 300 -N {read_count} -1 101 -2 101 {filename} frag1Copy_test.fastq frag2Copy_test.fastq"
result = subprocess.check_output(command, shell=True)
print(result)

[wgsim] seed = 1700252633
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 1 sequences, total length: 53173757


b'chr22\t48\tC\tS\t+\nchr22\t1943\tG\tR\t+\nchr22\t2502\tC\tG\t-\nchr22\t2990\tT\tA\t-\nchr22\t4864\tT\tK\t+\nchr22\t5438\tG\tK\t+\nchr22\t5768\tT\t-\t-\nchr22\t6185\tA\tR\t+\nchr22\t6680\tA\tR\t+\nchr22\t6682\tC\tG\t-\nchr22\t7519\tT\t-\t-\nchr22\t8574\tT\tC\t-\nchr22\t10988\tC\tA\t-\nchr22\t13859\tC\t-\t-\nchr22\t14446\tG\tA\t-\nchr22\t14464\tG\tK\t+\nchr22\t14555\tT\tA\t-\nchr22\t16639\tT\tW\t+\nchr22\t16746\tG\tR\t+\nchr22\t16875\tA\tM\t+\nchr22\t18174\t-\tA\t+\nchr22\t18469\tC\tM\t+\nchr22\t19178\tC\tS\t+\nchr22\t19627\tC\tG\t-\nchr22\t19729\tC\t-\t-\nchr22\t19745\tT\t-\t-\nchr22\t20078\tG\tC\t-\nchr22\t22866\tG\tR\t+\nchr22\t26118\tG\tR\t+\nchr22\t27144\tG\tS\t+\nchr22\t27958\tG\tT\t-\nchr22\t28298\tA\tC\t-\nchr22\t29230\t-\tAG\t+\nchr22\t29357\tA\tC\t-\nchr22\t30315\tA\tR\t+\nchr22\t30684\tC\tY\t+\nchr22\t32682\tT\tY\t+\nchr22\t34713\tG\tC\t-\nchr22\t35231\tT\tW\t+\nchr22\t36312\tC\tM\t+\nchr22\t37329\tA\tC\t-\nchr22\t37564\tG\tK\t+\nchr22\t38159\tT\tY\t+\nchr22\t39223\tG\tR\t+\

In [5]:
# bowtie related stuff
!bowtie2-build reference.fa bias
!bowtie2 -p 4 -k 15 --no-mixed -x bias -1 frag1Copy_test.fastq -2 frag2Copy_test.fastq -S biasOut.sam

Settings:
  Output files: "bias.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:01
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:00
bmax according to bmaxDivN setting: 12826141
Using parameters --bmax 9619606 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9619606 --dcv 1024
Constructing suffix-array element

In [6]:
# sam to bam file conversion
!samtools sort biasOut.sam -o biasOut_sorted.bam
!samtools index biasOut_sorted.bam

[bam_sort_core] merging from 9 files and 1 in-memory blocks...


In [7]:
app = cnvpytor.Root("test.pytor", create=True, max_cores=8)

In [8]:
app.rd(["biasOut_sorted.bam"])

In [9]:
app.calculate_histograms([100, 1000, 10000, 100000, 1000000, 10000000])

In [10]:
app.partition([100, 1000, 10000, 100000, 1000000, 10000000])

In [11]:
calls = app.call([100, 1000, 10000, 100000, 1000000, 10000000], print_calls=True)

deletion	chr22:1-200	200	0.4187	4.759669e+05	1.611881e-02	1.000000e+00	1.000000e+00	0.0000	1.0000	0
duplication	chr22:643001-644800	1800	1.7424	1.682966e-02	1.161400e+07	1.000000e+00	1.000000e+00	0.0000	1.0000	0
duplication	chr22:1306101-1325100	19000	1.6515	0.000000e+00	1.175636e-309	0.000000e+00	3.843913e-276	0.0000	1.0000	0
duplication	chr22:8039401-8047900	8500	2.6840	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:13480501-13513000	32500	6.9127	0.000000e+00	2.348576e-31	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:17447001-17810700	363700	1.3262	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0001	0.0000	599300
duplication	chr22:18662501-18665900	3400	1.9949	9.374854e-11	2.918734e-09	1.207663e+00	1.119998e+02	0.0000	0.0000	1814800
duplication	chr22:18680801-18682700	1900	1.9168	2.849420e-02	7.699308e+02	1.000000e+00	1.000000e+00	0.0000	0.0000	1826900
duplication	chr22:18704701-18881700	177000	4.6070	0.000000e+00	1.182956e-

In [12]:
for bin_size in calls:
    print("Bin size: ", bin_size)

Bin size:  100
Bin size:  1000
Bin size:  10000
Bin size:  100000
Bin size:  1000000
Bin size:  10000000


In [13]:
print(f"{filename=}, Histogram Size: 100")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_2_formated.fa', Histogram Size: 100
type,chromosome_name,start,end,size,cnv_level,q0
deletion,chr22,1,200,200,0.4186583861585439,0.0
duplication,chr22,643001,644800,1800,1.742407529148436,0.0
duplication,chr22,1306101,1325100,19000,1.6515190488629965,0.0
duplication,chr22,8039401,8047900,8500,2.6840454977360304,0.0
duplication,chr22,13480501,13513000,32500,6.912730302014416,0.0
duplication,chr22,17447001,17810700,363700,1.3262444658316093,7.913198989823626e-05
duplication,chr22,18662501,18665900,3400,1.9949044971843197,0.0
duplication,chr22,18680801,18682700,1900,1.9168375510854103,0.0
duplication,chr22,18704701,18881700,177000,4.60699836101312,0.00014652720488316965
duplication,chr22,20335901,20338300,2400,1.759602497730163,0.0
duplication,chr22,20338601,20340900,2300,1.6962813245087107,0.0016759776536312849
duplication,chr22,20341301,20344500,3200,1.9737998237562189,0.0012079378774805867
duplication,chr22,20345701,20347800,2100,1.6475507993483316,0.0
dupli

In [14]:
print(f"{filename=}, Histogram Size: 1000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_2_formated.fa', Histogram Size: 1000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,1306001,1325000,19000,1.6459816646997194,0.0
duplication,chr22,8039001,8048000,9000,2.5823730890256233,0.0
duplication,chr22,13481001,13513000,32000,6.976481592061167,0.0
duplication,chr22,17447001,17811000,364000,1.3192930638801283,7.908558981467986e-05
duplication,chr22,18658001,18683000,25000,1.5029867328078599,0.00037482340051321974
duplication,chr22,18705001,18881000,176000,4.603966783096031,0.00011681411651603911
duplication,chr22,20326001,20368000,42000,1.5430910058373084,0.00041847307544232604
duplication,chr22,20610001,20655000,45000,1.8977461992316114,0.00017187161826677023
duplication,chr22,21466001,21505000,39000,1.6145975066150533,0.0001897238655375222
duplication,chr22,21522001,21559000,37000,1.5636527854101425,0.001707397157465172
duplication,chr22,21561001,21686000,125000,1.764020172666079,0.0005363750885757027
duplication,chr22,23608001,23

In [15]:
print(f"{filename=}, Histogram Size: 10,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_2_formated.fa', Histogram Size: 10,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,17450001,17810000,360000,1.3132583346785078,7.994664132738835e-05
duplication,chr22,18650001,18880000,230000,3.7367465529717268,0.00016448393166440292
duplication,chr22,21460001,21680000,220000,1.6312719052802742,0.00045323568255492856
duplication,chr22,25150001,25310000,160000,1.6161536665003882,0.00011821188962742516
duplication,chr22,28460001,28600000,140000,4.952854340102096,4.707939076129729e-05
duplication,chr22,30800001,30980000,180000,5.070372052758137,1.5450779462110826e-05
duplication,chr22,35940001,36200000,260000,3.4866187970859377,3.3394716001793776e-05
duplication,chr22,36470001,36570000,100000,1.7884054310724578,7.236457271735001e-05
duplication,chr22,44150001,44460000,310000,3.5013803113192283,2.7992344093890322e-05
duplication,chr22,46380001,46670000,290000,2.67242461964298,2.6689479147790883e-05


In [16]:
print(f"{filename=}, Histogram Size: 100,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_2_formated.fa', Histogram Size: 100,000
type,chromosome_name,start,end,size,cnv_level,q0


In [17]:
print(f"{filename=}, Histogram Size: 1,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_2_formated.fa', Histogram Size: 1,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [18]:
print(f"{filename=}, Histogram Size: 10,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del_400_2_formated.fa', Histogram Size: 10,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [19]:
del app